In [1]:
import os
import glob

import mne
from joblib import Parallel, delayed

In [2]:
sfreq = 100
n_jobs = 30

In [3]:
files = glob.glob('/home/jovyan/competition/data/**/*_raw.fif.gz')

In [4]:
def do(file, sfreq):
    raw = mne.io.read_raw_fif(file, verbose='error')
    raw.resample(sfreq=sfreq)
    raw.save(file.replace('_raw.fif.gz', f'_{sfreq}_hz_raw.fif.gz'))

In [6]:
_ = Parallel(n_jobs=n_jobs)(delayed(do)(file, sfreq) for file in files)

Opening raw data file /home/jovyan/competition/data/testing/subj1393_EO_raw.fif.gz...
Opening raw data file /home/jovyan/competition/data/testing/subj1273_EO_raw.fif.gz...
Opening raw data file /home/jovyan/competition/data/testing/subj1293_EC_raw.fif.gz...
    Range : 18 ... 9997 =      0.036 ...    19.994 secs
Ready.
Opening raw data file /home/jovyan/competition/data/testing/subj1373_EC_raw.fif.gz...
    Range : 19 ... 9997 =      0.038 ...    19.994 secs
Ready.
Opening raw data file /home/jovyan/competition/data/testing/subj1541_EO_raw.fif.gz...
Opening raw data file /home/jovyan/competition/data/testing/subj1441_EC_raw.fif.gz...
Opening raw data file /home/jovyan/competition/data/testing/subj1306_EC_raw.fif.gz...
    Range : 11 ... 19997 =      0.022 ...    39.994 secs
Ready.
    Range : 8 ... 9997 =      0.016 ...    19.994 secs
Ready.
Opening raw data file /home/jovyan/competition/data/testing/subj1206_EO_raw.fif.gz...
    Range : 21 ... 19997 =      0.042 ...    39.994 secs
Rea

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [8]:
print("hi")

hi


In [ ]:
# download data https://filesender.renater.fr/?s=download&token=e1de0ec4-09bc-4194-b85b-59830cb04af3
# download test data from https://codalab.lisn.upsaclay.fr/competitions/8336

# Path to training data
train_path = "/home/jovyan/competition/data/training/"
# Path to testing data (public test set)
test_path = "/home/jovyan/competition/data/testing/"
train_subj = 1200  # use 10 instead of 1200 training subjects, for demonstration purpose
test_subj = 400  # use 10 instead of 400 testing subjects, for demonstration purpose

train_raws, test_raws = {}, {}
for condition in ["EC", "EO"]:
    train_raws[condition] = []
    test_raws[condition] = []
    train_subjs = list(range(1, train_subj + 1))
    for s in train_subjs:
        fname = f"subj{s:04}_{condition}{sfreq}_hz_raw.fif.gz"
        raw = mne.io.read_raw(train_path + fname, preload=False, verbose='error')
        train_raws[condition].append(raw)
    test_subjs = list(range(1201, 1201 + test_subj))
    for s in test_subjs:
        fname = f"subj{s:04}_{condition}{sfreq}_hz_raw.fif.gz"
        raw = mne.io.read_raw(test_path + fname, preload=False, verbose='error')
        test_raws[condition].append(raw)

In [ ]:
meta = pd.read_csv(train_path + "train_subjects.csv", index_col=0)
meta = pd.concat([meta, meta])
meta['condition'] = len(train_raws['EC']) * ['EC'] + len(train_raws['EO']) * ['EO']
train_raws = train_raws['EC'] + train_raws['EO']
len(train_raws), len(meta)

In [ ]:
test_meta = pd.DataFrame({'condition': len(test_raws['EC']) * ['EC'] + len(test_raws['EO']) * ['EO']})
test_raws = test_raws['EC'] + test_raws['EO']
len(test_raws), len(test_meta)

In [ ]:
from braindecode.datasets import BaseConcatDataset, BaseDataset

In [ ]:
train = BaseConcatDataset([
    BaseDataset(raw, target_name=target_name) for raw in train_raws
])
meta['subject'] = meta['id']
train.set_description(meta)

In [ ]:
with open(train_path + 'train.pkl', 'wb') as f:
    pickle.dump(train, f)

In [ ]:
test = BaseConcatDataset([
    BaseDataset(raw) for raw in test_raws
])
test_meta['subject'] = test_subjs + test_subjs
test.set_description(test_meta)

In [ ]:
with open(test_path + 'test.pkl', 'wb') as f:
    pickle.dump(test, f)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

In [ ]:
fig, ax_arr = plt.subplots(1, 2, figsize=(5,5), sharex=True, sharey=True)
bins = bins=range(5, 23)
ax = ax_arr[0]
ax = sns.histplot(data=train.description, y='age', bins=bins, ax=ax)
ax.set_yticks(bins);
ax.set_yticklabels(bins);
ax.set_title('Train ages')

ax = ax_arr[1]
ax = sns.histplot(data=test.description, y='age', bins=bins, ax=ax)
ax.set_title('Valid ages')